# Split the video into audio and muted video

In [1]:
import os

def extract_audio_and_remove(video_path, output_prefix):
    # Create a folder named 'temp-files' if it doesn't exist
    os.makedirs('temp-files', exist_ok=True)

    # Extract the audio
    audio_output_path = os.path.join('temp-files', 'audio.mp3')
    os.system(f"ffmpeg -i {video_path} -vn -ar 44100 -ac 2 -ab 192k -f mp3 {audio_output_path}")

    # Remove the audio from the video
    video_output_path = os.path.join('temp-files', 'video_without_audio.mp4')
    os.system(f"ffmpeg -i {video_path} -c:v copy -an {video_output_path}")

video_path = "original-video/video.mp4"
output_prefix = "output"

extract_audio_and_remove(video_path, output_prefix)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
from pydub import AudioSegment
from pydub.silence import detect_silence
import os

def save_segments(segments):
    # Ensure the 'segments' folder exists or create it
    output_folder = 'temp-files/segments'
    os.makedirs(output_folder, exist_ok=True)

    for i, segment in enumerate(segments):
        output_file = f"{output_folder}/segment_{i + 1}.wav"
        segment.export(output_file, format="wav")
        print(f"Segment {i + 1} exported to {output_file}")

def split_audio(input_file, silence_threshold=-40, silence_duration=500):
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Detect all silent sections
    silent_sections = detect_silence(audio, silence_thresh=silence_threshold, min_silence_len=silence_duration)

    # Store all segments, including silent periods
    all_segments = []
    all_segments.append(audio[:silent_sections[0][0]])
    for i, section in enumerate(silent_sections):
        start, end = section

        # If it's not the first section, store the non-silent period before this section
        if i > 0:
            prev_end = silent_sections[i-1][1]
            non_silent_period = audio[prev_end:start]
            all_segments.append(non_silent_period)

        # Store the silent section
        silence_period = audio[start:end]
        all_segments.append(silence_period)

    # If there's a non-silent section after the last silent section, store it
    if silent_sections and silent_sections[-1][1] < len(audio):
        last_end = silent_sections[-1][1]
        non_silent_period = audio[last_end:]
        all_segments.append(non_silent_period)

    return all_segments

# Example usage
input_file = "temp-files/audio.mp3"
all_segments = split_audio(input_file)

# Save segments to the current directory
save_segments(all_segments)

Segment 1 exported to temp-files/segments/segment_1.wav
Segment 2 exported to temp-files/segments/segment_2.wav
Segment 3 exported to temp-files/segments/segment_3.wav
Segment 4 exported to temp-files/segments/segment_4.wav
Segment 5 exported to temp-files/segments/segment_5.wav
Segment 6 exported to temp-files/segments/segment_6.wav
Segment 7 exported to temp-files/segments/segment_7.wav
Segment 8 exported to temp-files/segments/segment_8.wav
Segment 9 exported to temp-files/segments/segment_9.wav
Segment 10 exported to temp-files/segments/segment_10.wav
Segment 11 exported to temp-files/segments/segment_11.wav
Segment 12 exported to temp-files/segments/segment_12.wav
Segment 13 exported to temp-files/segments/segment_13.wav
Segment 14 exported to temp-files/segments/segment_14.wav
Segment 15 exported to temp-files/segments/segment_15.wav
Segment 16 exported to temp-files/segments/segment_16.wav
Segment 17 exported to temp-files/segments/segment_17.wav
Segment 18 exported to temp-file